\
            # 50. 工程基础：Docker（容器化）（Docker Fundamentals）

            目标：掌握 Docker 的核心概念与常用操作：镜像/容器、Dockerfile、端口/环境变量、卷、网络、Compose。
本章以“把一个后端服务容器化”为主线，并提供可选的本机 docker 检测示例。

            > 约定：Python 3.8；示例尽量只用标准库；代码块可直接运行（第三方依赖会做可选降级）。


## 前置知识

- 命令行基础
- 了解服务端应用的启动方式（例如 python app.py）


## 知识点地图

- 1. 核心概念：镜像 vs 容器、层（layers）与仓库
- 2. Dockerfile 基础：FROM/WORKDIR/COPY/RUN/CMD
- 3. 运行参数：端口映射、环境变量、卷（volume）
- 4. 网络与 Compose：多容器协作（概念）
- 5. 最佳实践：更小、更快、更安全
- 6. 可选：检测本机 docker 是否可用
- 7. 把一个服务容器化：最小模板（可复用）


## 自检清单（学完打勾）

- [ ] 理解镜像（image）与容器（container）的区别
- [ ] 会写 Dockerfile（FROM/WORKDIR/COPY/RUN/CMD）并构建镜像
- [ ] 会用端口映射、环境变量、卷（volume）配置运行
- [ ] 理解网络与 Compose 在多容器协作中的作用
- [ ] 知道常见最佳实践（分层缓存、最小镜像、非 root、.dockerignore）


In [ ]:
\
from pathlib import Path

ART = Path('_nb_artifacts')
ART.mkdir(exist_ok=True)
print('artifacts dir:', ART.resolve())


## 知识点 1：核心概念：镜像 vs 容器、层（layers）与仓库

- 镜像（image）：可分发的“只读模板”，由多层叠加。
- 容器（container）：镜像的“运行实例”，有自己的可写层。
- Registry：镜像仓库（Docker Hub/私有仓库）。

理解层缓存很重要：合理拆分 Dockerfile 可以显著加速构建。


## 知识点 2：Dockerfile 基础：FROM/WORKDIR/COPY/RUN/CMD

典型 Dockerfile：
- FROM：选择基础镜像
- WORKDIR：工作目录
- COPY：拷贝代码
- RUN：构建时执行（安装依赖/编译）
- CMD：容器启动时执行（默认命令）

注意：RUN 在构建期执行；CMD 在运行期执行。


## 知识点 3：运行参数：端口映射、环境变量、卷（volume）

- 端口：`-p 8080:8080` 把容器端口暴露给宿主
- 环境变量：`-e KEY=VALUE` 或 `--env-file`
- 卷：
  - bind mount：把本地目录挂进容器（开发常用）
  - named volume：由 docker 管理（数据持久化）


## 知识点 4：网络与 Compose：多容器协作（概念）

当你需要“应用 + 数据库 + 中间件”时：
- Compose 用一个 `docker-compose.yml` 描述多个服务
- 默认网络下服务可互相用服务名访问（DNS）

常见组合：web + mysql + redis + rabbitmq。


## 知识点 5：最佳实践：更小、更快、更安全

- 使用更小基础镜像（但注意兼容性）
- 利用层缓存：先 COPY 依赖清单再 RUN install
- 写 `.dockerignore` 避免把垃圾文件打进镜像
- 尽量非 root 运行
- 健康检查、日志、时区、只读文件系统（进阶）


## 知识点 6：可选：检测本机 docker 是否可用

如果本机装了 docker，本段会打印版本信息；否则给出提示。


In [ ]:
import shutil
import subprocess


docker = shutil.which('docker')
if not docker:
    print('docker not found. Please install Docker Desktop (Windows) and ensure it is in PATH.')
else:
    p = subprocess.run([docker, '--version'], text=True, capture_output=True)
    print(p.stdout.strip() or p.stderr.strip())


## 知识点 7：把一个服务容器化：最小模板（可复用）

下面给出一个“Python HTTP 服务”的容器化模板（文本）。你可以把它套到自己的 REST 服务上。

说明：
- 生产建议用 gunicorn/uvicorn（框架相关）
- 这里为了通用性只展示模板与要点


In [ ]:
from pathlib import Path

app_py = r"""
from http.server import BaseHTTPRequestHandler, HTTPServer
import json

class H(BaseHTTPRequestHandler):
    def do_GET(self):
        body = json.dumps({'ok': True, 'path': self.path}).encode('utf-8')
        self.send_response(200)
        self.send_header('Content-Type', 'application/json; charset=utf-8')
        self.send_header('Content-Length', str(len(body)))
        self.end_headers()
        self.wfile.write(body)

    def log_message(self, fmt, *args):
        return

if __name__ == '__main__':
    server = HTTPServer(('0.0.0.0', 8080), H)
    server.serve_forever()
"""

dockerfile = r"""
FROM python:3.11-slim
WORKDIR /app
COPY app.py /app/app.py
EXPOSE 8080
CMD ["python", "app.py"]
"""

art = Path('_nb_artifacts')
art.mkdir(exist_ok=True)
(art / 'docker_demo').mkdir(exist_ok=True)
(art / 'docker_demo' / 'app.py').write_text(app_py, encoding='utf-8')
(art / 'docker_demo' / 'Dockerfile').write_text(dockerfile.strip() + "\n", encoding='utf-8')

print('wrote:', (art / 'docker_demo' / 'app.py'))
print('wrote:', (art / 'docker_demo' / 'Dockerfile'))
print('build:  docker build -t demo-http .')
print('run:    docker run --rm -p 8080:8080 demo-http')


## 常见坑

- 把开发环境当生产：镜像里包含调试依赖/源码泄露/.env
- Dockerfile 层不合理：每次改代码都重新安装依赖
- 不写 .dockerignore：把 node_modules/缓存/密钥打进镜像
- 容器内用 localhost 访问“另一个容器”：应该用服务名/网络
- 数据不持久化：没用 volume 导致容器删了数据也没了


## 综合小案例：为“应用 + MySQL + RabbitMQ”写一份 Compose（设计题）

写一个 compose 方案（不必真的跑起来）：
- web：你的 Python 服务（端口映射、环境变量）
- mysql：挂载 volume，初始化密码与库
- rabbitmq：管理端口映射（可选）

同时写出：
- 容器之间如何互相访问（服务名）
- 关键环境变量（MYSQL_*/AMQP_URL）
- 数据持久化与备份策略（简述）


In [ ]:
# 建议输出 docker-compose.yml（文本）并写下注释说明。
# 本 cell 不运行代码。


## 自测题（不写代码也能回答）

- 镜像与容器的区别是什么？
- RUN 与 CMD 的区别是什么？
- 为什么 .dockerignore 很重要？
- 多容器环境里，服务之间应该怎么互相访问？


## 练习题（建议写代码）

- 把 Dockerfile 改成：先 COPY requirements.txt 再 pip install（利用缓存）。
- 为镜像增加非 root 用户并以该用户运行（了解）。
- 写一个 compose：web + redis，用 redis 做缓存（与缓存章呼应）。
